# PETase Prediction Pipeline - Google Drive Version

## 📁 Folder Structure
```
/content/drive/MyDrive/PET2025/pipeline/
├── resources/              ← Input files here
│   ├── petase_pipeline_ndcg_optimized.py
│   ├── predictive-pet-zero-shot-test-2025.csv
│   └── pet-2025-wildtype-cds.csv
├── submission/             ← Output files here
│   └── (results will appear here)
└── driver.ipynb            ← This notebook
```

## 🔄 Disconnect Recovery
**If you disconnect:**
1. Just re-run all cells
2. Pipeline will automatically resume from last checkpoint
3. All progress is saved to Google Drive

## ⚡ Quick Start
1. Run Cell 1: Mount Google Drive
2. Run Cell 2: Install dependencies
3. Run Cell 3: Prepare data
4. Run Cell 4: Run pipeline (2-3 hours)
5. Run Cell 5: Check results

In [3]:
# ========================================
# CELL 1: Mount Google Drive
# ========================================

from google.colab import drive
import os

print("📂 Mounting Google Drive...")
drive.mount('/content/drive', force_remount=False)
print("✅ Google Drive mounted")

# Define paths
BASE_DIR = '/content/drive/MyDrive/PET2025/pipeline'
RESOURCES_DIR = f'{BASE_DIR}/resources'
SUBMISSION_DIR = f'{BASE_DIR}/submission'

# Verify directories exist
print("\n🔍 Verifying folder structure...")
if not os.path.exists(BASE_DIR):
    print(f"❌ Base directory not found: {BASE_DIR}")
    print("   Please create it first!")
else:
    print(f"✅ Base directory: {BASE_DIR}")

if not os.path.exists(RESOURCES_DIR):
    print(f"⚠️  Resources directory not found: {RESOURCES_DIR}")
    print("   Creating it...")
    os.makedirs(RESOURCES_DIR, exist_ok=True)
else:
    print(f"✅ Resources directory: {RESOURCES_DIR}")

if not os.path.exists(SUBMISSION_DIR):
    print(f"⚠️  Submission directory not found: {SUBMISSION_DIR}")
    print("   Creating it...")
    os.makedirs(SUBMISSION_DIR, exist_ok=True)
else:
    print(f"✅ Submission directory: {SUBMISSION_DIR}")

# Check required files in resources
print("\n📋 Checking required files in resources/...")
required_files = [
    'petase_pipeline_ndcg_optimized.py',
    'predictive-pet-zero-shot-test-2025.csv',
    'pet-2025-wildtype-cds.csv'
]

missing_files = []
for filename in required_files:
    filepath = f'{RESOURCES_DIR}/{filename}'
    if os.path.exists(filepath):
        size = os.path.getsize(filepath) / 1024  # KB
        print(f"   ✅ {filename} ({size:.1f} KB)")
    else:
        print(f"   ❌ {filename} - NOT FOUND")
        missing_files.append(filename)

if missing_files:
    print(f"\n⚠️  Missing {len(missing_files)} file(s). Please upload them to:")
    print(f"   {RESOURCES_DIR}/")
    print(f"\n   Missing: {missing_files}")
else:
    print("\n🎉 All required files found!")

# Change to base directory
os.chdir(BASE_DIR)
print(f"\n📍 Working directory: {os.getcwd()}")

ModuleNotFoundError: No module named 'google'

In [ ]:
# ========================================
# CELL 2: Install Dependencies
# ========================================

print("📦 Installing dependencies...")
print("   This may take 1-2 minutes...\n")

# Install ESM and other packages
!pip install esm pandas numpy scipy tqdm torch -q

print("\n✅ Dependencies installed!")

# Verify imports
print("\n🔍 Verifying installations...")
try:
    import pandas as pd
    import numpy as np
    from scipy.stats import gmean
    import torch
    from esm.sdk import client as esm_client
    from esm.sdk.api import ESMProtein, LogitsConfig
    print("   ✅ All packages imported successfully")
except ImportError as e:
    print(f"   ❌ Import error: {e}")
    print("   Please re-run this cell")

📦 Installing dependencies...
   This may take 1-2 minutes...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 155.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 146.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.6 MB/s eta 0:00:

In [ ]:
# ========================================
# CELL 3: Prepare Data (Fix CDS file)
# ========================================

import pandas as pd
import shutil

print("🔧 Preparing data files...\n")

# Copy pipeline script to working directory
print("📋 Copying pipeline script...")
pipeline_src = f'{RESOURCES_DIR}/petase_pipeline_ndcg_optimized.py'
pipeline_dst = 'petase_pipeline_ndcg_optimized.py'
shutil.copy(pipeline_src, pipeline_dst)
print(f"   ✅ Copied to working directory")

# Copy test data to working directory
print("\n📋 Copying test data...")
test_src = f'{RESOURCES_DIR}/predictive-pet-zero-shot-test-2025.csv'
test_dst = 'predictive-pet-zero-shot-test-2025.csv'
shutil.copy(test_src, test_dst)
print(f"   ✅ Copied test data")

# Fix CDS file column names
print("\n🔧 Fixing CDS file column names...")
cds_src = f'{RESOURCES_DIR}/pet-2025-wildtype-cds.csv'
cds_df = pd.read_csv(cds_src)

print(f"   Original columns: {cds_df.columns.tolist()}")

# Rename columns
cds_df = cds_df.rename(columns={
    'Wt AA Sequence': 'sequence',
    'CDS': 'cds'
})

print(f"   Fixed columns: {cds_df.columns.tolist()}")

# Verify
print("\n   Verification:")
print(f"     Protein sequence length: {len(cds_df['sequence'].iloc[0])} aa")
print(f"     DNA sequence length: {len(cds_df['cds'].iloc[0])} bp")
print(f"     Number of sequences: {len(cds_df)}")

# Save fixed version
cds_fixed = 'pet-2025-wildtype-cds-fixed.csv'
cds_df.to_csv(cds_fixed, index=False)
print(f"\n✅ Saved fixed CDS file: {cds_fixed}")

print("\n🎉 Data preparation complete!")
print("\n📁 Working directory files:")
import os
for f in os.listdir('.'):
    if f.endswith(('.py', '.csv')):
        size = os.path.getsize(f) / 1024
        print(f"   • {f} ({size:.1f} KB)")

🔧 Preparing data files...

📋 Copying pipeline script...
   ✅ Copied to working directory

📋 Copying test data...
   ✅ Copied test data

🔧 Fixing CDS file column names...
   Original columns: ['Wt AA Sequence', 'CDS']
   Fixed columns: ['sequence', 'cds']

   Verification:
     Protein sequence length: 259 aa
     DNA sequence length: 777 bp
     Number of sequences: 313

✅ Saved fixed CDS file: pet-2025-wildtype-cds-fixed.csv

🎉 Data preparation complete!

📁 Working directory files:
   • petase_pipeline_ndcg_optimized.py (36.6 KB)
   • predictive-pet-zero-shot-test-2025.csv (1274.2 KB)
   • pet-2025-wildtype-cds-fixed.csv (301.5 KB)


In [ ]:
# ========================================
# CELL 4: Run Pipeline
# ========================================
# ⚠️  This cell takes 2-3 hours to complete!
#
# Features:
# - Auto-saves checkpoint every 100 sequences
# - If disconnected, re-run this cell to resume
# - All progress saved to Google Drive
# ========================================

import os
from pathlib import Path

print("="*70)
print(" "*15 + "🚀 STARTING PETASE PIPELINE")
print("="*70)

# Your ESM3 token
ESM3_TOKEN = "4KmDJvB1gQaRJJDrWoBa91"

# Check if checkpoint exists
checkpoint_file = 'phase1_checkpoint.pkl'
if os.path.exists(checkpoint_file):
    import pickle
    with open(checkpoint_file, 'rb') as f:
        checkpoint = pickle.load(f)
    last_idx = checkpoint.get('last_index', -1)
    print(f"\n📂 Checkpoint found!")
    print(f"   Last processed: {last_idx + 1} / 4988 sequences")
    print(f"   Progress: {100*(last_idx+1)/4988:.1f}%")
    print(f"\n🔄 Pipeline will resume from sequence {last_idx + 2}")
else:
    print(f"\n📝 No checkpoint found - starting from beginning")

print(f"\n⏱️  Estimated time: 2-3 hours")
print(f"💾 Checkpoints saved every 100 sequences")
print(f"📁 Final outputs will be in: {SUBMISSION_DIR}/")
print(f"\n{'='*70}\n")

# Run pipeline
!python petase_pipeline_ndcg_optimized.py \
    --input predictive-pet-zero-shot-test-2025.csv \
    --cds pet-2025-wildtype-cds-fixed.csv \
    --output submission \
    --token {ESM3_TOKEN}

print("\n" + "="*70)
print(" "*20 + "✅ PIPELINE COMPLETE!")
print("="*70)

               🚀 STARTING PETASE PIPELINE

📝 No checkpoint found - starting from beginning

⏱️  Estimated time: 2-3 hours
💾 Checkpoints saved every 100 sequences
📁 Final outputs will be in: /content/drive/MyDrive/PET2025/pipeline/submission/



          PETase Prediction Pipeline - NDCG@10 Optimized

📁 Input: predictive-pet-zero-shot-test-2025.csv
📁 CDS: pet-2025-wildtype-cds-fixed.csv
📁 Output prefix: submission
🎯 Strategy: all

📂 Loading data...
✅ Loaded 4988 sequences
✅ Loaded 313 wildtype CDS

🔍 Detecting column names...
   Test CSV columns: ['sequence', 'activity_1 (μmol [TPA]/min·mg [E])', 'activity_2 (μmol [TPA]/min·mg [E])', 'expression (mg/mL)']
   CDS CSV columns: ['sequence', 'cds']
✅ Column names detected and standardized
🧬 Initializing ESM3 client...
✅ ESM3 client initialized successfully

PHASE 1: ESM3 Evolutionary Features
Processing sequences:  18% 914/4988 [32:56<2:50:55,  2.52s/it]Retrying... Attempt 1 after 1.0s due to: (502, 'Failure in encode: {"status":"error","m

In [ ]:
# ========================================
# CELL 5: Move Results to Submission Folder
# ========================================

import shutil
import os
from datetime import datetime

print("📦 Moving results to submission folder...\n")

# List of output files to move
output_files = [
    'submission_balanced.csv',
    'submission_conservative.csv',
    'submission_aggressive.csv',
    'submission_diverse.csv',
    'submission_base_only.csv',
    'phase1_complete.csv',
    'phase2_complete.csv',
    'phase3_complete.csv'
]

# Add timestamp to folder name
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
run_dir = f'{SUBMISSION_DIR}/run_{timestamp}'
os.makedirs(run_dir, exist_ok=True)

print(f"📁 Creating directory: {run_dir}\n")

moved_count = 0
for filename in output_files:
    src = filename
    if os.path.exists(src):
        dst = f'{run_dir}/{filename}'
        shutil.copy(src, dst)
        size = os.path.getsize(dst) / 1024  # KB
        print(f"   ✅ {filename} ({size:.1f} KB)")
        moved_count += 1
    else:
        print(f"   ⚠️  {filename} - not found (skipped)")

print(f"\n✅ Moved {moved_count} files to submission folder")
print(f"\n📂 Results location:")
print(f"   {run_dir}")

# Also copy to main submission folder (latest)
print(f"\n📋 Copying submission files to main folder...")
submission_files = ['submission_balanced.csv', 'submission_conservative.csv']
for filename in submission_files:
    src = filename
    if os.path.exists(src):
        dst = f'{SUBMISSION_DIR}/{filename}'
        shutil.copy(src, dst)
        print(f"   ✅ {filename}")

print(f"\n🎉 All results saved!")
print(f"\n📊 Access your submissions at:")
print(f"   Main: {SUBMISSION_DIR}/")
print(f"   This run: {run_dir}/")

In [ ]:
# ========================================
# CELL 6: Check Results
# ========================================

import pandas as pd
import os

print("="*70)
print(" "*20 + "📊 RESULTS SUMMARY")
print("="*70)

# Check submission files
submission_files = {
    'Balanced (Recommended)': 'submission_balanced.csv',
    'Conservative': 'submission_conservative.csv',
    'Aggressive': 'submission_aggressive.csv',
    'Diverse': 'submission_diverse.csv',
    'Base Only': 'submission_base_only.csv'
}

print("\n📋 Generated Submission Files:\n")

for strategy, filename in submission_files.items():
    filepath = f'{SUBMISSION_DIR}/{filename}'
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        size = os.path.getsize(filepath) / 1024
        print(f"✅ {strategy:25} {filename}")
        print(f"   Sequences: {len(df):,}   Size: {size:.1f} KB")
        print(f"   Columns: {list(df.columns)}")
        print(f"   Activity 1: mean={df['activity_1'].mean():.2f}, range=[{df['activity_1'].min():.2f}, {df['activity_1'].max():.2f}]")
        print(f"   Activity 2: mean={df['activity_2'].mean():.2f}, range=[{df['activity_2'].min():.2f}, {df['activity_2'].max():.2f}]")
        print(f"   Expression: mean={df['expression'].mean():.2f}, range=[{df['expression'].min():.2f}, {df['expression'].max():.2f}]")
        print()
    else:
        print(f"❌ {strategy:25} {filename} - NOT FOUND")
        print()

print("="*70)
print("\n💡 Recommendations:")
print("   1st submission: submission_balanced.csv")
print("   2nd submission: submission_conservative.csv")
print("   3rd submission: submission_aggressive.csv or submission_diverse.csv")
print("\n📂 All files are in: " + SUBMISSION_DIR)
print("="*70)

# 🔄 Disconnect Recovery Instructions

## If Colab Disconnects During Phase 1:

### Option 1: Quick Resume (Recommended)
1. Just re-run Cell 4 (Run Pipeline)
2. Pipeline will automatically detect checkpoint
3. It will resume from last saved position

### Option 2: Full Restart
If you want to start fresh:
1. Delete checkpoint file:
   ```python
   import os
   if os.path.exists('phase1_checkpoint.pkl'):
       os.remove('phase1_checkpoint.pkl')
       print('Checkpoint deleted')
   ```
2. Re-run Cell 4

## How Checkpointing Works:
- Pipeline saves progress every 100 sequences
- Checkpoint file: `phase1_checkpoint.pkl`
- Stored in Google Drive (persists across sessions)
- Contains: processed sequences + computed features

## Progress Tracking:
You'll see output like:
```
Processing sequences: 45% 2244/4988 [1:15:00<1:22:00, 1.8s/it]
```

This means:
- 45% complete (2244 out of 4988 sequences)
- 1:15:00 elapsed
- 1:22:00 remaining
- 1.8 seconds per sequence

## Tips for Avoiding Disconnects:
1. Keep browser tab active (don't minimize)
2. Use Colab Pro+ for more stable connections
3. Don't close laptop (if applicable)
4. Check internet connection is stable

## Checking Progress After Reconnect:
```python
import pickle
with open('phase1_checkpoint.pkl', 'rb') as f:
    checkpoint = pickle.load(f)
print(f"Last processed: {checkpoint['last_index'] + 1} / 4988")
```

In [ ]:
# ========================================
# CELL 7: Check Checkpoint Status (Optional)
# ========================================
# Run this anytime to check progress

import os
import pickle

checkpoint_file = 'phase1_checkpoint.pkl'

if os.path.exists(checkpoint_file):
    try:
        with open(checkpoint_file, 'rb') as f:
            checkpoint = pickle.load(f)

        last_idx = checkpoint.get('last_index', -1)
        total = 4988

        print("📊 Checkpoint Status\n")
        print(f"   Last processed index: {last_idx}")
        print(f"   Completed sequences: {last_idx + 1} / {total}")
        print(f"   Progress: {100*(last_idx+1)/total:.1f}%")
        print(f"   Remaining: {total - (last_idx + 1)} sequences")

        # Estimate time remaining (assuming 2s per sequence)
        remaining = total - (last_idx + 1)
        est_seconds = remaining * 2
        est_hours = est_seconds / 3600
        est_minutes = (est_seconds % 3600) / 60

        print(f"\n   Estimated time remaining: {int(est_hours)}h {int(est_minutes)}m")
        print(f"   (assuming 2 seconds per sequence)")

        # Check if any features are computed
        df = checkpoint.get('df')
        if df is not None:
            print(f"\n   DataFrame shape: {df.shape}")
            print(f"   Columns: {len(df.columns)}")

    except Exception as e:
        print(f"❌ Error reading checkpoint: {e}")
else:
    print("📝 No checkpoint file found")
    print("   Either pipeline hasn't started, or Phase 1 is complete")

    # Check if Phase 1 complete file exists
    if os.path.exists('phase1_complete.csv'):
        import pandas as pd
        df = pd.read_csv('phase1_complete.csv')
        print(f"\n✅ Phase 1 complete!")
        print(f"   Sequences processed: {len(df)}")
        print(f"   Features computed: {len(df.columns)}")

In [ ]:
# ========================================
# CELL 8: Download Results to Local (Optional)
# ========================================
# Only needed if you want to download files to your computer

from google.colab import files
import os

print("📥 Downloading submission files...\n")

# Download main submission files
download_files = [
    f'{SUBMISSION_DIR}/submission_balanced.csv',
    f'{SUBMISSION_DIR}/submission_conservative.csv'
]

for filepath in download_files:
    if os.path.exists(filepath):
        filename = os.path.basename(filepath)
        print(f"Downloading {filename}...")
        files.download(filepath)
    else:
        print(f"❌ {filepath} not found")

print("\n✅ Downloads complete!")
print("\n💡 Note: Files are already saved in Google Drive at:")
print(f"   {SUBMISSION_DIR}/")